# Adding DDD

In [17]:
from my_rem import *
from my_ddd import *
from typing import List, Callable
import numpy as np
from matplotlib import pyplot as plt

import qiskit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService

from mitiq.interface.mitiq_qiskit import to_qiskit
from mitiq import ddd, QPROGRAM
from mitiq.ddd import insert_ddd_sequences

In [86]:
n_qubits = 2
shots = 100000
error_probability = 0.1
readout_noise = get_readout_noise(error_probability)

mitigator = generate_mitigator(n_qubits, shots, readout_noise)

States:
['00', '01', '10', '11']
00 becomes {'11': 990, '01': 8991, '10': 8912, '00': 81107}
01 becomes {'11': 9004, '00': 8942, '10': 982, '01': 81072}
10 becomes {'01': 975, '11': 8903, '10': 81093, '00': 9029}
11 becomes {'01': 9081, '00': 995, '10': 9013, '11': 80911}

Confusion Matrix:
[[0.811070 0.089910 0.089120 0.009900]
 [0.089420 0.810720 0.009820 0.090040]
 [0.090290 0.009750 0.810930 0.089030]
 [0.009950 0.090810 0.090130 0.809110]] 

Mitigator:
[[1.263669 -0.140200 -0.138892 0.015423]
 [-0.139427 1.264510 0.015652 -0.140734]
 [-0.140755 0.015994 1.263891 -0.139129]
 [0.015788 -0.141979 -0.140839 1.267030]]


In [87]:
import cirq

def rep_ixix_rule(window_length: int) -> Callable[[int], QPROGRAM]:
    return ddd.rules.repeated_rule(
        window_length, [cirq.I, cirq.X, cirq.I, cirq.X]
    )

def rep_xx_rule(window_length: int) -> Callable[[int], QPROGRAM]:
    return ddd.rules.repeated_rule(window_length, [cirq.X, cirq.X])

# Set DDD sequences to test.
rules = [rep_ixix_rule, rep_xx_rule, ddd.rules.xx]

# Test the sequence insertion
for rule in rules:
    print(rule(10))

0: ───I───I───X───I───X───I───X───I───X───I───
0: ───X───X───X───X───X───X───X───X───X───X───
0: ───I───I───I───X───I───I───X───I───I───I───


In [88]:
# Total number of shots to use.
shots = 10000

# Test at multiple depths.
depths = [10, 30, 50, 100]

In [89]:
num_qubits = 2  # Number of qubits in the circuit
num_operations = 5  # Number of initial random operations
idle_depth = 20  # Number of idle operations to add
circuit = create_random_circuit_with_idle_windows(num_qubits, num_operations, idle_depth)
print(circuit)

          ┌───┐┌───┐     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_0: ──■──┤ X ├┤ X ├─────┤ X ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├»
     ┌─┴─┐└───┘└─┬─┘┌───┐└─┬─┘├───┤├───┤├───┤├───┤├───┤├───┤├───┤└┬─┬┘└───┘»
q_1: ┤ X ├───────■──┤ H ├──■──┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├┤ I ├─┤M├──────»
     └───┘          └───┘     └───┘└───┘└───┘└───┘└───┘└───┘└───┘ └╥┘      »
c: 2/══════════════════════════════════════════════════════════════╩═══════»
                                                                   1       »
«     ┌───┐┌───┐┌───┐┌───┐┌─┐
«q_0: ┤ I ├┤ I ├┤ I ├┤ I ├┤M├
«     └───┘└───┘└───┘└───┘└╥┘
«q_1: ─────────────────────╫─
«                          ║ 
«c: 2/═════════════════════╩═
«                          0 


In [90]:
ixix_circ = insert_ddd_sequences(circuit, rep_ixix_rule)
print(ixix_circ)

          ┌───┐┌───┐     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_0: ──■──┤ X ├┤ X ├─────┤ X ├┤ I ├┤ I ├┤ X ├┤ I ├┤ X ├┤ I ├┤ X ├┤ I ├┤ X ├»
     ┌─┴─┐└───┘└─┬─┘┌───┐└─┬─┘├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤»
q_1: ┤ X ├───────■──┤ H ├──■──┤ I ├┤ I ├┤ X ├┤ I ├┤ X ├┤ I ├┤ X ├┤ I ├┤ X ├»
     └───┘          └───┘     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
c: 2/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐┌───┐┌───┐┌───┐┌─┐   
«q_0: ┤ I ├┤ X ├┤ I ├┤ X ├┤M├───
«     ├───┤├───┤├───┤├───┤└╥┘┌─┐
«q_1: ┤ I ├┤ X ├┤ I ├┤ X ├─╫─┤M├
«     └───┘└───┘└───┘└───┘ ║ └╥┘
«c: 2/═════════════════════╩══╩═
«                          0  1 


In [91]:
# define noise model for circuit and combine it with readout error
from qiskit_aer.noise import depolarizing_error

def get_combined_noise(p_read, p_circ):
    """Create a noise model with a bit-flip error during measurement."""
    error_meas = pauli_error([('X', p_read), ('I', 1 - p_read)])
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")

    # Single-qubit depolarizing error
    error_depolarizing_1q = depolarizing_error(p_circ, 1)
    noise_model.add_all_qubit_quantum_error(error_depolarizing_1q, ['u1', 'u2', 'u3'])
    
    # Two-qubit depolarizing error
    error_depolarizing_2q = depolarizing_error(p_circ, 2)
    noise_model.add_all_qubit_quantum_error(error_depolarizing_2q, 'cx')

    return noise_model

In [92]:
shots = 100000
noise = get_combined_noise(0.1,0.01)
simulator = AerSimulator(noise_model=noise)
counts = simulator.run(ixix_circ,shots=shots).result().get_counts(ixix_circ)
# Normalize the counts to get probabilities
probabilities = {k: v / shots for k, v in counts.items()}

print(probabilities)

{'11': 0.09569, '00': 0.09344, '10': 0.4047, '01': 0.40617}


In [93]:
# REM PART
# Sort the probabilities dictionary by its keys (binary strings converted to integers)
print("Empirical probabilities:", (probability_vector:=np.array([probability for state, probability in sorted(probabilities.items(), key=lambda x: int(x[0], 2))])))

print("Mitigated quasi-probabilities:", (mitigated_probabilities := np.dot(mitigator, probability_vector)))

print("Closest positive probabilities:", (closest_probabilities := closest_positive_distribution(mitigated_probabilities)))

Empirical probabilities: [0.093440 0.406170 0.404700 0.095690]
Mitigated quasi-probabilities: [0.006399 0.493445 0.491527 0.008052]
Closest positive probabilities: [0.006536 0.493603 0.491677 0.008184]


In [ ]:
# TO DO: combine error models previously defined, so that no mistakes are made (namely, the readout error flipping probability needs to be the same
# when creating the mitigator and when running the circuit with general noise)